<a href="https://colab.research.google.com/github/pascalghanimi/Ski-Classification-AI/blob/main/Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import joblib

# Lade die Daten
result = joblib.load(f"/content/4D-Humans/outputs/results/demo_{video_name}.pkl")
print(list(result.keys())) # Packt alles in eine Liste und zeigt die Framenamen als Liste an
frame_key = list(result.keys())[0]  # Nimm den ersten Frame
joints_3d = np.array(result[frame_key]['3d_joints'][0])  # 3D-Joints der ersten Person

# Verbindungen aus dem Paper
connections = [
    (2, 3), # Rechte Schulter bis rechter Ellbogen
    (3, 4), # Rechter Ellbogen bis rechtes Handgelenk
    (5, 6),
    (6, 7),
    (9, 10), # Rechte Hüfte bis zum rechten Knie
    (10, 11), # Rechtes Knie bis zum rechten Knöchel
    (12, 13),
    (13, 14),
]

# Berechne die Segmentlängen
segment_lengths = []
for start, end in connections:
    p1 = joints_3d[start]  # Koordinaten des ersten Joints
    p2 = joints_3d[end]    # Koordinaten des zweiten Joints
    length = np.linalg.norm(p2 - p1)  # Euklidische Distanz
    segment_lengths.append(length)


print(segment_lengths, "Segmentlängen")

# Ausgabe der Segmentlängen
for i, length in enumerate(segment_lengths):
    print(f"Segment {connections[i]}: Länge = {length:.4f}")


In [ ]:
# Der gesamte Code dient der Berechnung des MPJPE
import re

# Hier definiere ich die Gelenkspunkte (an welchen Positionen sie sich in der result[frame]['3d_joints'][0] Liste befinden )


right_shoulder = 2
left_shoulder = 5
right_ellbow = 3
left_ellbow = 6
right_wrist = 4
left_wrist = 7
right_hip = 9
left_hip = 12
right_knee = 10
left_knee = 13
right_ankle = 11
left_ankle = 14

# Hier erstelle ich ein Dictionary, um die Punkte alle zu speichern
_4D_humans_joint_positions = {}

# Hier iteriere ich über alle Frames und speiche jeden einzelnen Punkt für jeden einzelnen Frame in einem weiteren Dictionary, das den Key i hat (was der Framenummer entspricht)
for i, frame in enumerate(list(result.keys())):
    # Das Zentrum bei 4D Humans mit den Koordinaten 0, 0, 0 ist der 44 Punkt, für einen guten Vergleich mit dem anderen Modell braucht man allerdings den Mittelpunkt zwischen der Hüfte
    center = (result[frame]['3d_joints'][0][right_hip] + result[frame]['3d_joints'][0][left_hip]) / 2
    # center = result[frame]['3d_joints'][0][44] # dies wäre ein Nullpunkt des Koordinatensystems mit 0 0 0
    if (i < 5):
      print(center, "Center")

    _4D_humans_joint_positions[i] = {
        "center": center,
        "right_shoulder": result[frame]['3d_joints'][0][right_shoulder] - center,
        "left_shoulder": result[frame]['3d_joints'][0][left_shoulder] - center,
        "right_ellbow": result[frame]['3d_joints'][0][right_ellbow] - center,
        "left_ellbow": result[frame]['3d_joints'][0][left_ellbow] - center,
        "right_wrist": result[frame]['3d_joints'][0][right_wrist] - center,
        "left_wrist": result[frame]['3d_joints'][0][left_wrist] - center,
        "right_hip": result[frame]['3d_joints'][0][right_hip] - center,
        "left_hip": result[frame]['3d_joints'][0][left_hip] - center,
        "right_knee": result[frame]['3d_joints'][0][right_knee] - center,
        "left_knee": result[frame]['3d_joints'][0][left_knee] - center,
        "right_ankle": result[frame]['3d_joints'][0][right_ankle] - center,
        "left_ankle": result[frame]['3d_joints'][0][left_ankle] - center,
        "right_shoulder_ellbow_length": np.linalg.norm(result[frame]['3d_joints'][0][right_shoulder] - result[frame]['3d_joints'][0][right_ellbow]),
        "right_ellbow_wrist_length": np.linalg.norm(result[frame]['3d_joints'][0][right_ellbow] - result[frame]['3d_joints'][0][right_wrist]),
        "right_hip_knee_length": np.linalg.norm(result[frame]['3d_joints'][0][right_hip] - result[frame]['3d_joints'][0][right_knee]),
        "right_knee_ankle_length": np.linalg.norm(result[frame]['3d_joints'][0][right_knee] - result[frame]['3d_joints'][0][right_ankle]),
        "left_shoulder_ellbow_length": np.linalg.norm(result[frame]['3d_joints'][0][left_shoulder] - result[frame]['3d_joints'][0][left_ellbow]),
        "left_ellbow_wrist_length": np.linalg.norm(result[frame]['3d_joints'][0][left_ellbow] - result[frame]['3d_joints'][0][left_wrist]),
        "left_hip_knee_length": np.linalg.norm(result[frame]['3d_joints'][0][left_hip] - result[frame]['3d_joints'][0][left_knee]),
        "left_knee_ankle_length": np.linalg.norm(result[frame]['3d_joints'][0][left_knee] - result[frame]['3d_joints'][0][left_ankle])
    }


In [ ]:
# Hier werden die Vergleichsdaten von manuell erstellen Modell importiert und mit data.head() visualisiert

import pandas as pd
import numpy as np

txt_file = "/content/4D-Humans/Simon_PD.txt"

data = pd.read_csv(f"{txt_file}", delim_whitespace=True)
data.columns = data.columns.str.strip()  # Entferne Leerzeichen am Anfang/Ende
data.columns = data.columns.str.replace(r'\t', '', regex=True)  # Entferne Tabs
data.columns = data.columns.str.replace(r'\s+', '_', regex=True)  # Ersetze Leerzeichen durch '_'

data.head()


In [ ]:
# Right side
shoulder_right = data[['sh_r_X', 'sh_r_Y', 'sh_r_Z']].values
elbow_right = data[['el_r_X', 'el_r_Y', 'el_r_Z']].values
wrist_right = data[['wr_r_X', 'wr_r_Y', 'wr_r_Z']].values
hip_right = data[['hip_r_X', 'hip_r_Y', 'hip_r_Z']].values
knee_right = data[['kn_r_X', 'kn_r_Y', 'kn_r_Z']].values
ankle_right = data[['an_r_X', 'an_r_Y', 'an_r_Z']].values

# Left side
shoulder_left = data[['sh_l_X', 'sh_l_Y', 'sh_l_Z']].values
elbow_left = data[['el_l_X', 'el_l_Y', 'el_l_Z']].values
wrist_left = data[['wr_l_X', 'wr_l_Y', 'wr_l_Z']].values
hip_left = data[['hip_l_X', 'hip_l_Y', 'hip_l_Z']].values
knee_left = data[['kn_l_X', 'kn_l_Y', 'kn_l_Z']].values
ankle_left = data[['an_l_X', 'an_l_Y', 'an_l_Z']].values

ref_center = (hip_right + hip_left) / 2

In [ ]:
# Euklidischer Abstand zwischen dem Zentrum und der rechten Schulter Referenzmodell
for i in range(3):
  diff = np.linalg.norm(shoulder_right[i] - ref_center[i])
  print("Referenzmodell Schulter: ", diff)


# Euklidischer Abstand zwischen dem Zentrum und der rechten Schulter 4D Humans
for i in range(3):
    print("4D Humans Schulter", np.linalg.norm(_4D_humans_joint_positions[i]["right_shoulder"] - _4D_humans_joint_positions[i]["center"]))

# Euklidischer Abstand zwischen dem Zentrum und dem rechten Fußgelenk Referenzmodell
for i in range(3):
  diff = np.linalg.norm(ankle_right[i] - ref_center[i])
  print("Referenzmodell Ankle: ", diff)


# Euklidischer Abstand zwischen dem Zentrum und dem rechten Fußgelenk 4D Humans
for i in range(3):
    print("4D Humans Ankle", np.linalg.norm(_4D_humans_joint_positions[i]["right_ankle"] - _4D_humans_joint_positions[i]["center"]))


# Es gibt hier wirklich große Unterschiede und unabhängig davon, ob man den Mittelpunkt zwischen Hüfte Links und Hüfte Rechts verwendet, es gibt immer Punkte,
# für die die Segmentlängen komplett anders sind (für die Schulter funktioniert es beispielsweise ganz gut mit dem Mittelpunkt der Hüftknochen, für das Ankle allerdings gar nicht)

In [ ]:
comparison_joint_positions = {}

# Gelenkpositionen aus der CSV-Datei
comparison_joint_positions = {}
for i, frame in enumerate(data["Frame"]):
    # Berechnung des Mittelpunkts pro Frame
    center = (hip_right[i] + hip_left[i]) / 2

    if (i < 5):
      print("Center: ", center)
      print("Abstand zwischen Schulter und Zentrum", np.linalg.norm(shoulder_right[i] - center))

    comparison_joint_positions[i] = {
        "right_shoulder": shoulder_right[i] - center,
        "left_shoulder": shoulder_left[i] - center,
        "right_ellbow": elbow_right[i] - center,
        "left_ellbow": elbow_left[i] - center,
        "right_wrist": wrist_right[i] - center,
        "left_wrist": wrist_left[i] - center,
        "right_hip": hip_right[i] - center,
        "left_hip": hip_left[i] - center,
        "right_knee": knee_right[i] - center,
        "left_knee": knee_left[i] - center,
        "right_ankle": ankle_right[i] - center,
        "left_ankle": ankle_left[i] - center,
        "right_shoulder_ellbow_length": np.linalg.norm(shoulder_right[i] - elbow_right[i]),
        "right_ellbow_wrist_length": np.linalg.norm(elbow_right[i] - wrist_right[i]),
        "right_hip_knee_length": np.linalg.norm(hip_right[i] - knee_right[i]),
        "right_knee_ankle_length": np.linalg.norm(knee_right[i] - ankle_right[i]),
        "left_shoulder_ellbow_length": np.linalg.norm(shoulder_left[i] - elbow_left[i]),
        "left_ellbow_wrist_length": np.linalg.norm(elbow_left[i] - wrist_left[i]),
        "left_hip_knee_length": np.linalg.norm(hip_left[i] - knee_left[i]),
        "left_knee_ankle_length": np.linalg.norm(knee_left[i] - ankle_left[i])
    }



for i in range(5):
  print("Referenz", comparison_joint_positions[i]["right_shoulder"])
  print("4D Humans", _4D_humans_joint_positions[i]["right_shoulder"])

In [ ]:
# Abstand exemplarisch zwischen einigen Gelenkspunkten der beiden Modelle -> sehr schlechtes Ergebnis
right_shoulder_diff = comparison_joint_positions[0]["right_shoulder"] - _4D_humans_joint_positions[0]["right_shoulder"]
right_shoulder_diff_euclidic = np.linalg.norm(comparison_joint_positions[0]["right_shoulder"] - _4D_humans_joint_positions[0]["right_shoulder"])
print(right_shoulder_diff)
print(right_shoulder_diff_euclidic)

In [ ]:
# Alle möglichen Rotationen der Achsen (mit ChatGPT) -> alle von ChatGPT vorgeschlagenen Rotationen führen immer noch zu sehr schlechten Ergebnissen (hab die Rotationen nicht selber geprüft)
rotations = [
    ("Original", lambda p: p),  # Keine Änderung
    ("[x, y, z] -> [z, y, x]", lambda p: [p[2], p[1], p[0]]),
    ("[x, y, z] -> [y, x, z]", lambda p: [p[1], p[0], p[2]]),
    ("[x, y, z] -> [x, z, y]", lambda p: [p[0], p[2], p[1]]),
    ("[x, y, z] -> [y, z, x]", lambda p: [p[1], p[2], p[0]]),
    ("[x, y, z] -> [z, x, y]", lambda p: [p[2], p[0], p[1]])
]

# Spiegelung (optional)
def flip(p):
    return [p[0], p[1], -p[2]]

print("Vergleich für rechte Schulter:\n")
for name, rotate in rotations:
    # Drehe und spiegele Referenzdaten
    rotated_ref = rotate(comparison_joint_positions[0]["right_shoulder"])
    rotated_and_flipped_ref = flip(rotated_ref)

    # Berechne den Unterschied zum 4D Humans Modell
    diff_original = np.linalg.norm(np.array(rotated_ref) - _4D_humans_joint_positions[0]["right_shoulder"])
    diff_flipped = np.linalg.norm(np.array(rotated_and_flipped_ref) - _4D_humans_joint_positions[0]["right_shoulder"])

    # Ausgabe der Ergebnisse
    print(f"Rotation: {name}")
    print(f"  Abstand (nur rotiert): {diff_original:.6f}")
    print(f"  Abstand (rotiert und gespiegelt): {diff_flipped:.6f}")
    print("-" * 50)

In [ ]:
# Position Error für die Schulter -> die Werte sind so schlecht, dass es sich eig. nicht lohnt, den MPJPE zu benutzen
right_shoulder_differences = []
print(len(_4D_humans_joint_positions))
print(len(comparison_joint_positions))

for i, frame in enumerate(list(result.keys())):
  right_shoulder_difference = np.linalg.norm(_4D_humans_joint_positions[i]["right_shoulder"] - comparison_joint_positions[i]["right_shoulder"])
  right_shoulder_differences.append(right_shoulder_difference)

print(right_shoulder_differences)
print(np.mean(right_shoulder_differences))

In [ ]:
# Length differences of segments between both models on the first frame
difference_between_shoulder_ellbow_right = comparison_joint_positions[0]["right_shoulder_ellbow_length"] - _4D_humans_joint_positions[0]["right_shoulder_ellbow_length"]
difference_between_ellbow_wrist_right = comparison_joint_positions[0]["right_ellbow_wrist_length"] - _4D_humans_joint_positions[0]["right_ellbow_wrist_length"]
difference_between_hip_knee_right = comparison_joint_positions[0]["right_hip_knee_length"] - _4D_humans_joint_positions[0]["right_hip_knee_length"]
difference_between_knee_ankle_right = comparison_joint_positions[0]["right_knee_ankle_length"] - _4D_humans_joint_positions[0]["right_knee_ankle_length"]

difference_between_shoulder_ellbow_left = comparison_joint_positions[0]["left_shoulder_ellbow_length"] - _4D_humans_joint_positions[0]["left_shoulder_ellbow_length"]
difference_between_ellbow_wrist_left = comparison_joint_positions[0]["left_ellbow_wrist_length"] - _4D_humans_joint_positions[0]["left_ellbow_wrist_length"]
difference_between_hip_knee_left = comparison_joint_positions[0]["left_hip_knee_length"] - _4D_humans_joint_positions[0]["left_hip_knee_length"]
difference_between_knee_ankle_left = comparison_joint_positions[0]["left_knee_ankle_length"] - _4D_humans_joint_positions[0]["left_knee_ankle_length"]

print("Length differences of segments between both models on the first frame")

print(difference_between_shoulder_ellbow_right, "Shoulder Ellbow Right")
print(difference_between_ellbow_wrist_right, "Ellbow Wrist Right")
print(difference_between_hip_knee_right, "Hip Knee Right")
print(difference_between_knee_ankle_right, "Knee Ankle Right")

print(difference_between_shoulder_ellbow_left, "Shoulder Ellbow Left")
print(difference_between_ellbow_wrist_left, "Ellbow Wrist Left")
print(difference_between_hip_knee_left, "Hip Knee Left")
print(difference_between_knee_ankle_left, "Knee Ankle Left")

In [ ]:
# Mean length differences of segments between both models on all frames
mean_diff_shoulder_ellbow_right = []
mean_diff_ellbow_wrist_right = []
mean_diff_hip_knee_right = []
mean_diff_knee_ankle_right = []

mean_diff_shoulder_ellbow_left = []
mean_diff_ellbow_wrist_left = []
mean_diff_hip_knee_left = []
mean_diff_knee_ankle_left = []

for (i, frame) in enumerate(_4D_humans_joint_positions):
    # Right side
    mean_diff_shoulder_ellbow_right.append(np.abs(comparison_joint_positions[i]["right_shoulder_ellbow_length"] - _4D_humans_joint_positions[i]["right_shoulder_ellbow_length"]))
    mean_diff_ellbow_wrist_right.append(np.abs(comparison_joint_positions[i]["right_ellbow_wrist_length"] - _4D_humans_joint_positions[i]["right_ellbow_wrist_length"]))
    mean_diff_hip_knee_right.append(np.abs(comparison_joint_positions[i]["right_hip_knee_length"] - _4D_humans_joint_positions[i]["right_hip_knee_length"]))
    mean_diff_knee_ankle_right.append(np.abs(comparison_joint_positions[i]["right_knee_ankle_length"] - _4D_humans_joint_positions[i]["right_knee_ankle_length"]))

    # Left side
    mean_diff_shoulder_ellbow_left.append(np.abs(comparison_joint_positions[i]["left_shoulder_ellbow_length"] - _4D_humans_joint_positions[i]["left_shoulder_ellbow_length"]))
    mean_diff_ellbow_wrist_left.append(np.abs(comparison_joint_positions[i]["left_ellbow_wrist_length"] - _4D_humans_joint_positions[i]["left_ellbow_wrist_length"]))
    mean_diff_hip_knee_left.append(np.abs(comparison_joint_positions[i]["left_hip_knee_length"] - _4D_humans_joint_positions[i]["left_hip_knee_length"]))
    mean_diff_knee_ankle_left.append(np.abs(comparison_joint_positions[i]["left_knee_ankle_length"] - _4D_humans_joint_positions[i]["left_knee_ankle_length"]))

# Output mean differences
print("Right Shoulder-Elbow: ", np.mean(mean_diff_shoulder_ellbow_right))
print("Right Elbow-Wrist: ", np.mean(mean_diff_ellbow_wrist_right))
print("Right Hip-Knee: ", np.mean(mean_diff_hip_knee_right))
print("Right Knee-Ankle: ", np.mean(mean_diff_knee_ankle_right))

print("Left Shoulder-Elbow: ", np.mean(mean_diff_shoulder_ellbow_left))
print("Left Elbow-Wrist: ", np.mean(mean_diff_ellbow_wrist_left))
print("Left Hip-Knee: ", np.mean(mean_diff_hip_knee_left))
print("Left Knee-Ankle: ", np.mean(mean_diff_knee_ankle_left))


In [ ]:
# Standardabweichungen zwischen den Frames bei den einzelnen Punkten sowie bei den Segmentlängen
r_shoulder_ellbow = []
r_ellbow_wrist = []
r_hip_knee = []
r_knee_ankle = []
l_shoulder_ellbow = []
l_ellbow_wrist = []
l_hip_knee = []
l_knee_ankle = []


for (i, frame) in enumerate(_4D_humans_joint_positions):
  r_shoulder_ellbow.append(_4D_humans_joint_positions[i]["right_shoulder_ellbow_length"])
  l_shoulder_ellbow.append(_4D_humans_joint_positions[i]["left_shoulder_ellbow_length"])
  r_ellbow_wrist.append(_4D_humans_joint_positions[i]["right_ellbow_wrist_length"])
  l_ellbow_wrist.append(_4D_humans_joint_positions[i]["left_ellbow_wrist_length"])
  r_hip_knee.append(_4D_humans_joint_positions[i]["right_hip_knee_length"])
  l_hip_knee.append(_4D_humans_joint_positions[i]["left_hip_knee_length"])
  r_knee_ankle.append(_4D_humans_joint_positions[i]["right_knee_ankle_length"])
  l_knee_ankle.append(_4D_humans_joint_positions[i]["left_knee_ankle_length"])


std_4d_humans = {
    "r_shoulder_ellbow": np.std(r_shoulder_ellbow),
    "r_ellbow_wrist": np.std(r_ellbow_wrist),
    "r_hip_knee": np.std(r_hip_knee),
    "r_knee_ankle": np.std(r_knee_ankle),
    "l_shoulder_ellbow": np.std(l_shoulder_ellbow),
    "l_ellbow_wrist": np.std(l_ellbow_wrist),
    "l_hip_knee": np.std(l_hip_knee),
    "l_knee_ankle": np.std(l_knee_ankle)
}

print(std_4d_humans)

In [ ]:
r_shoulder_ellbow_ref = []
r_ellbow_wrist_ref = []
r_hip_knee_ref = []
r_knee_ankle_ref = []
l_shoulder_ellbow_ref = []
l_ellbow_wrist_ref = []
l_hip_knee_ref = []
l_knee_ankle_ref = []

for (i, frame) in enumerate(comparison_joint_positions):
    r_shoulder_ellbow_ref.append(comparison_joint_positions[i]["right_shoulder_ellbow_length"])
    l_shoulder_ellbow_ref.append(comparison_joint_positions[i]["left_shoulder_ellbow_length"])
    r_ellbow_wrist_ref.append(comparison_joint_positions[i]["right_ellbow_wrist_length"])
    l_ellbow_wrist_ref.append(comparison_joint_positions[i]["left_ellbow_wrist_length"])
    r_hip_knee_ref.append(comparison_joint_positions[i]["right_hip_knee_length"])
    l_hip_knee_ref.append(comparison_joint_positions[i]["left_hip_knee_length"])
    r_knee_ankle_ref.append(comparison_joint_positions[i]["right_knee_ankle_length"])
    l_knee_ankle_ref.append(comparison_joint_positions[i]["left_knee_ankle_length"])

std_reference_model = {
    "r_shoulder_ellbow": np.std(r_shoulder_ellbow_ref),
    "r_ellbow_wrist": np.std(r_ellbow_wrist_ref),
    "r_hip_knee": np.std(r_hip_knee_ref),
    "r_knee_ankle": np.std(r_knee_ankle_ref),
    "l_shoulder_ellbow": np.std(l_shoulder_ellbow_ref),
    "l_ellbow_wrist": np.std(l_ellbow_wrist_ref),
    "l_hip_knee": np.std(l_hip_knee_ref),
    "l_knee_ankle": np.std(l_knee_ankle_ref)
}

print(std_reference_model)


In [ ]:
import matplotlib.pyplot as plt

# Daten der Segmentlängen (4D Humans und Referenzmodell)
data = [
    r_shoulder_ellbow, r_shoulder_ellbow_ref,  # Right Shoulder-Elbow
    r_ellbow_wrist, r_ellbow_wrist_ref,        # Right Elbow-Wrist
    r_hip_knee, r_hip_knee_ref,                # Right Hip-Knee
    r_knee_ankle, r_knee_ankle_ref,            # Right Knee-Ankle
    l_shoulder_ellbow, l_shoulder_ellbow_ref,  # Left Shoulder-Elbow
    l_ellbow_wrist, l_ellbow_wrist_ref,        # Left Elbow-Wrist
    l_hip_knee, l_hip_knee_ref,                # Left Hip-Knee
    l_knee_ankle, l_knee_ankle_ref             # Left Knee-Ankle
]

# Labels entsprechend 4D Humans und Referenzmodell
labels = [
    "R Shoulder-Elbow (4D)", "R Shoulder-Elbow (Ref)",
    "R Elbow-Wrist (4D)", "R Elbow-Wrist (Ref)",
    "R Hip-Knee (4D)", "R Hip-Knee (Ref)",
    "R Knee-Ankle (4D)", "R Knee-Ankle (Ref)",
    "L Shoulder-Elbow (4D)", "L Shoulder-Elbow (Ref)",
    "L Elbow-Wrist (4D)", "L Elbow-Wrist (Ref)",
    "L Hip-Knee (4D)", "L Hip-Knee (Ref)",
    "L Knee-Ankle (4D)", "L Knee-Ankle (Ref)"
]

# Boxplot erstellen
plt.figure(figsize=(16, 8))
plt.boxplot(data, patch_artist=True, showmeans=True, labels=labels)

# Plot-Details
plt.title("Boxplot der Segmentlängen: 4D Humans vs Referenzmodell")
plt.ylabel("Länge")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Plot anzeigen
plt.tight_layout()
plt.show()


In [ ]:
# Histogramm zeigt an, wie die einzelnen Datenpunkte verteilt sind - bei 4D Human sind die Werte sehr einheitlich mit wenig Streuung, während das Referenzmodell eine größere Streuung aufweist
plt.hist(r_shoulder_ellbow, bins=20, alpha=0.7, label="4D Humans")
plt.hist(r_shoulder_ellbow_ref, bins=20, alpha=0.7, label="Referenzmodell")
plt.legend()
plt.title("Histogram Right Shoulder-Elbow")
plt.show()


In [ ]:
from typing_extensions import dataclass_transform
# Hier wird der COM berechnet

HAND_FOREARM_SCALE_FACTOR = 0.75

# Definiere die Segmente und relative Faktoren als Dictionary
segments = {
    "head_and_neck": {"points": (43, 40), "factor": 0.567}, # 43 ist Head, 40 ist Thorax
    "torso": {"points": (40, 8), "factor": 0.562}, # 40 ist Thorax, 8 ist Mid Hip
    "upper_arm_right": {"points": (2, 3), "factor": 0.564},  # 2 ist rechte Schulter, 3 ist rechter Ellbogen
    "upper_arm_left": {"points": (5, 6), "factor": 0.564},  # 5 ist linke Schulter, 6 ist linker Ellbogen
    "lower_arm_right": {"points": (3, 4), "factor": 0.57}, # 3 ist rechter Ellbogen, 4 ist rechtes Handgelenk
    "lower_arm_left": {"points": (6, 7), "factor": 0.57}, # 6 ist linker Ellbogen, 7 ist linkes Handgelenk
    "hand_right": {"factor": 0.468}, # Faktor ist hier 46.8, weil wir bei der Berechnung später bei der Wrist anfangen und die Länge des Ellbogens hinzuaddieren, hier also in die andere Richtung gehen als sonst
    "hand_left": {"factor": 0.468},
    "thigh_right": {"points": (9, 10), "factor": 0.567},  # 9 ist rechte Hüfte, 10 ist rechtes Knie
    "thigh_left": {"points": (12, 13), "factor": 0.567}, # 12 ist linke Hüfte, 13 ist linkes Knie
    "calf_right": {"points": (10, 11), "factor": 0.567}, # 10 ist rechtes Knie, 11 ist rechtes Ankle
    "calf_left": {"points": (13, 14), "factor": 0.567}, # 13 ist linkes Knie, 14 ist linkes Ankle
    "foot_right": {"points": (11, 22), "factor": 0.5}, # 11 ist rechtes Ankle, 22 ist großer Zeh rechts
    "foot_left": {"points": (14, 19), "factor": 0.5} # 14 ist linkes Ankle, 19 ist großer Zeh links
}


com_of_segments = {}

for i, frame in enumerate(list(result.keys())):
  com_of_segment = {}
  for segment, data in segments.items():
    if (segment == "hand_right"):
      com_position = np.array(result[frame]["3d_joints"][0][4]) + (
      (np.array(result[frame]["3d_joints"][0][4]) - np.array(result[frame]["3d_joints"][0][3]))
      * HAND_FOREARM_SCALE_FACTOR
      ) * data["factor"]
      com_of_segment[segment] = com_position
    elif segment == "hand_left":
      com_position = np.array(result[frame]["3d_joints"][0][7]) + (
      (np.array(result[frame]["3d_joints"][0][7]) - np.array(result[frame]["3d_joints"][0][6]))
      * HAND_FOREARM_SCALE_FACTOR
      ) * data["factor"]
      com_of_segment[segment] = com_position
    else:
      point_higher, point_lower = data["points"]
      com_position = np.array(result[frame]["3d_joints"][0][point_lower]) + ((np.array(result[frame]["3d_joints"][0][point_higher]) - np.array(result[frame]["3d_joints"][0][point_lower])) * data["factor"])
      com_of_segment[segment] = com_position


  com_of_segments[i] = com_of_segment


for i in range(3):
  print(com_of_segments[i]["head_and_neck"], "Head and Neck COM")

for i in range(3):
    print(com_of_segments[i]["lower_arm_right"], "Lower Arm Right COM")

for i in range(3):
  print(com_of_segments[i]["hand_right"], "Hand Right COM")



In [ ]:
relative_masses = {
    "head_and_neck": 0.082,  # Beispiel aus Tabelle Hall
    "torso": 0.4684,
    "upper_arm_right": 0.0325,
    "upper_arm_left": 0.0325,
    "lower_arm_right": 0.0196,  # Unterarmgewicht, basierend auf Literaturwerten
    "lower_arm_left": 0.0196,
    "hand_right": 0.0061,  # Handgewicht, basierend auf Literaturwerten
    "hand_left": 0.0061,
    "thigh_right": 0.105,
    "thigh_left": 0.105,
    "calf_right": 0.0475,
    "calf_left": 0.0475,
    "foot_right": 0.0143,
    "foot_left": 0.0143
}


# Liste, die die COMs für alle Frames speichert
COM_xyz_values = {}

for frame in com_of_segments:
  COM_xyz = np.sum([
    np.array(com_of_segments[frame][segment]) * relative_masses[segment]
    for segment in relative_masses.keys()
  ], axis=0)
  COM_xyz_values[frame] = COM_xyz

print(COM_xyz_values)


In [ ]:
r_ankle = {}
l_ankle = {}
center_ankle = {}

for i, frame in enumerate(list(result.keys())):
  r_ankle[i] = result[frame]["3d_joints"][0][11]
  l_ankle[i] = result[frame]["3d_joints"][0][14]
  center_ankle[i] = (result[frame]["3d_joints"][0][11] + result[frame]["3d_joints"][0][14]) / 2

print(len(r_ankle))
print(len(COM_xyz_values))


In [ ]:
COM_to_ground = {}

for i in COM_xyz_values:
  distance = np.linalg.norm(COM_xyz_values[i] - center_ankle[i])
  COM_to_ground[i] = distance

print(COM_to_ground)

In [ ]:
import matplotlib.pyplot as plt

# COM_to_ground-Daten, die du bereits berechnet hast
frames = list(COM_to_ground.keys())
distances = list(COM_to_ground.values())

# Plot erstellen
plt.figure(figsize=(10, 6))
plt.plot(frames, distances, marker='o', linestyle='-', color='b', label="COM to Ground Distance")
plt.title("Körperschwerpunkt (COM) Abstand zum Boden")
plt.xlabel("Frames")
plt.ylabel("Abstand (m)")
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend()
plt.show()


In [ ]:
# Hier wird das Feature Kniewinkel für jeden Frame berechnet
r_hip_num = 9
r_knee_num = 10
r_ankle_num = 11
r_shoulder_num = 2
l_hip_num = 12
l_knee_num = 13
l_ankle_num = 14
l_shoulder_num = 5

knee_angles_right = {}
knee_angles_left = {}

for i, frame in enumerate(list(result.keys())):
  vector_right_hip_knee = result[frame]['3d_joints'][0][r_knee_num] - result[frame]['3d_joints'][0][r_hip_num]
  vector_left_hip_knee = result[frame]['3d_joints'][0][l_knee_num] - result[frame]['3d_joints'][0][l_hip_num]
  vector_right_knee_ankle = result[frame]['3d_joints'][0][r_ankle_num] - result[frame]['3d_joints'][0][r_knee_num]
  vector_left_knee_ankle = result[frame]['3d_joints'][0][l_ankle_num] - result[frame]['3d_joints'][0][l_knee_num]
  right_leg_dot_product = np.dot(vector_right_hip_knee, vector_right_knee_ankle)
  left_leg_dot_product = np.dot(vector_left_hip_knee, vector_left_knee_ankle)

  knee_angle_right = np.arccos(right_leg_dot_product / (_4D_humans_joint_positions[i]["right_hip_knee_length"] * _4D_humans_joint_positions[i]["right_knee_ankle_length"]))
  knee_angle_left = np.arccos(right_leg_dot_product / (_4D_humans_joint_positions[i]["left_hip_knee_length"] * _4D_humans_joint_positions[i]["left_knee_ankle_length"]))

  knee_angles_right[i] = knee_angle_right
  knee_angles_left[i] = knee_angle_left


for i in range(5):
  print(knee_angles_right)


In [ ]:
import matplotlib.pyplot as plt

# Frames extrahieren
frames = list(knee_angles_right.keys())

# Kniewinkel extrahieren
right_knee_angles = [np.degrees(knee_angles_right[frame]) for frame in frames]  # Umwandlung in Grad
left_knee_angles = [np.degrees(knee_angles_left[frame]) for frame in frames]    # Umwandlung in Grad

# Plot erstellen
plt.figure(figsize=(12, 6))

# Kniewinkel (rechts) plotten
plt.plot(frames, right_knee_angles, label="Rechter Kniewinkel", linestyle='-', marker='o')

# Kniewinkel (links) plotten
plt.plot(frames, left_knee_angles, label="Linker Kniewinkel", linestyle='-', marker='o')

# Plot-Details
plt.title("Kniewinkel über die Frames")
plt.xlabel("Frames")
plt.ylabel("Kniewinkel (Grad)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.7)
plt.show()


In [ ]:
#exemplarische Darstellung des rechten Knies

import matplotlib.pyplot as plt

x_coords = [result[frame]['3d_joints'][0][r_knee_num][0] for frame in result.keys()]
y_coords = [result[frame]['3d_joints'][0][r_knee_num][1] for frame in result.keys()]
z_coords = [result[frame]['3d_joints'][0][r_knee_num][2] for frame in result.keys()]

plt.figure(figsize=(12, 6))
plt.plot(x_coords, label="x-Koordinate")
plt.plot(y_coords, label="y-Koordinate")
plt.plot(z_coords, label="z-Koordinate")
plt.title("Bewegung des rechten Knies")
plt.xlabel("Frames")
plt.ylabel("Position")
plt.legend()
plt.grid(True)
plt.show()

# x-Achse: Die Bewegung des Knies entlang der x-Achse ist logisch, da der Fahrer nach rechts und links fährt. Die starke Veränderung in den Werten passt zu einer Schwungbewegung.
# y-Achse: Diese sollte stabil bleiben, da das Knie (wie du sagst) nicht groß um die y-Achse rotiert. Die minimalen Änderungen könnten durch Tracking-Rauschen entstehen.
# z-Achse: Leichte Schwankungen in z sind plausibel, da das Knie aufgrund von Hoch- und Tiefbewegungen während des Fahrens minimal variiert.

In [ ]:
# Hier wird der Winkel zwischen den Gelenkspunkten und der y-Achse berechnet

z_vector = np.array([0, 0, 1])  # Z-Vektor für die Berechnung
angles_z = {}

for i, frame in enumerate(list(result.keys())):
    # Dictionary für die Winkel eines Frames
    angle_dic = {}

    # Gelenkpunkte des aktuellen Frames
    r_knee = result[frame]['3d_joints'][0][r_knee_num]
    l_knee = result[frame]['3d_joints'][0][l_knee_num]
    r_shoulder = result[frame]['3d_joints'][0][r_shoulder_num]
    l_shoulder = result[frame]['3d_joints'][0][l_shoulder_num]
    r_ankle = result[frame]['3d_joints'][0][r_ankle_num]
    l_ankle = result[frame]['3d_joints'][0][l_ankle_num]
    r_hip = result[frame]['3d_joints'][0][r_hip_num]
    l_hip = result[frame]['3d_joints'][0][l_hip_num]

    # Berechnung der Winkel zur z-Achse für jeden Gelenkpunkt
    angle_dic["angle_z_r_knee"] = np.degrees(np.arccos(np.dot(r_knee, z_vector) / (np.linalg.norm(r_knee) * np.linalg.norm(z_vector))))
    angle_dic["angle_z_l_knee"] = np.degrees(np.arccos(np.dot(l_knee, z_vector) / (np.linalg.norm(l_knee) * np.linalg.norm(z_vector))))
    angle_dic["angle_z_r_shoulder"] = np.degrees(np.arccos(np.dot(r_shoulder, z_vector) / (np.linalg.norm(r_shoulder) * np.linalg.norm(z_vector))))
    angle_dic["angle_z_l_shoulder"] = np.degrees(np.arccos(np.dot(l_shoulder, z_vector) / (np.linalg.norm(l_shoulder) * np.linalg.norm(z_vector))))
    angle_dic["angle_z_r_ankle"] = np.degrees(np.arccos(np.dot(r_ankle, z_vector) / (np.linalg.norm(r_ankle) * np.linalg.norm(z_vector))))
    angle_dic["angle_z_l_ankle"] = np.degrees(np.arccos(np.dot(l_ankle, z_vector) / (np.linalg.norm(l_ankle) * np.linalg.norm(z_vector))))
    angle_dic["angle_z_r_hip"] = np.degrees(np.arccos(np.dot(r_hip, z_vector) / (np.linalg.norm(r_hip) * np.linalg.norm(z_vector))))
    angle_dic["angle_z_l_hip"] = np.degrees(np.arccos(np.dot(l_hip, z_vector) / (np.linalg.norm(l_hip) * np.linalg.norm(z_vector))))

    # Speichern des Frame-Dictionaries im übergeordneten Dictionary
    angles_z[i] = angle_dic

# Ausgabe des finalen Dictionaries
print(angles_z[0])


In [ ]:
# Hier wird der Winkel zwischen den Gelenkspunkten und der y-Achse berechnet

z_vector = np.array([1, 0, 0])  # Z-Vektor für die Berechnung
angles_x = {}

for i, frame in enumerate(list(result.keys())):
    # Dictionary für die Winkel eines Frames
    angle_dic = {}

    # Gelenkpunkte des aktuellen Frames
    r_knee = result[frame]['3d_joints'][0][r_knee_num]
    l_knee = result[frame]['3d_joints'][0][l_knee_num]
    r_shoulder = result[frame]['3d_joints'][0][r_shoulder_num]
    l_shoulder = result[frame]['3d_joints'][0][l_shoulder_num]
    r_ankle = result[frame]['3d_joints'][0][r_ankle_num]
    l_ankle = result[frame]['3d_joints'][0][l_ankle_num]
    r_hip = result[frame]['3d_joints'][0][r_hip_num]
    l_hip = result[frame]['3d_joints'][0][l_hip_num]

    # Berechnung der Winkel zur z-Achse für jeden Gelenkpunkt
    angle_dic["angle_z_r_knee"] = np.degrees(np.arccos(np.dot(r_knee, z_vector) / (np.linalg.norm(r_knee) * np.linalg.norm(z_vector))))
    angle_dic["angle_z_l_knee"] = np.degrees(np.arccos(np.dot(l_knee, z_vector) / (np.linalg.norm(l_knee) * np.linalg.norm(z_vector))))
    angle_dic["angle_z_r_shoulder"] = np.degrees(np.arccos(np.dot(r_shoulder, z_vector) / (np.linalg.norm(r_shoulder) * np.linalg.norm(z_vector))))
    angle_dic["angle_z_l_shoulder"] = np.degrees(np.arccos(np.dot(l_shoulder, z_vector) / (np.linalg.norm(l_shoulder) * np.linalg.norm(z_vector))))
    angle_dic["angle_z_r_ankle"] = np.degrees(np.arccos(np.dot(r_ankle, z_vector) / (np.linalg.norm(r_ankle) * np.linalg.norm(z_vector))))
    angle_dic["angle_z_l_ankle"] = np.degrees(np.arccos(np.dot(l_ankle, z_vector) / (np.linalg.norm(l_ankle) * np.linalg.norm(z_vector))))
    angle_dic["angle_z_r_hip"] = np.degrees(np.arccos(np.dot(r_hip, z_vector) / (np.linalg.norm(r_hip) * np.linalg.norm(z_vector))))
    angle_dic["angle_z_l_hip"] = np.degrees(np.arccos(np.dot(l_hip, z_vector) / (np.linalg.norm(l_hip) * np.linalg.norm(z_vector))))

    # Speichern des Frame-Dictionaries im übergeordneten Dictionary
    angles_x[i] = angle_dic

# Ausgabe des finalen Dictionaries
print(angles_x[0])


In [ ]:
import matplotlib.pyplot as plt

def plot_right_angles(angles, axis):
    """
    Plottet die Winkel der rechten Körperteile über alle Frames.

    Parameter:
    - angles (dict): Ein Dictionary, bei dem die Schlüssel die Frame-Nummern sind
                     und die Werte wiederum Dictionaries mit Winkelinformationen enthalten.
    """
    # Sortiere die Frames für eine geordnete Darstellung
    sorted_frames = sorted(angles.keys())

    # Identifiziere alle Winkel-Keys für die rechten Körperteile
    right_angle_keys = [key for key in angles[0].keys() if key.startswith('angle_z_r_')]

    # Initialisiere ein Dictionary, um die Winkelwerte zu speichern
    angle_values = {key: [] for key in right_angle_keys}

    # Sammle die Winkelwerte für jeden Frame
    for frame in sorted_frames:
        for key in right_angle_keys:
            angle_values[key].append(angles[frame][key])

    # Erstelle das Plot-Fenster
    plt.figure(figsize=(12, 6))

    # Plotte jeden Winkel als eigene Linie
    for key in right_angle_keys:
        plt.plot(sorted_frames, angle_values[key], marker='o', linestyle='-', label=key)

    # Beschrifte die Achsen und füge einen Titel hinzu
    plt.xlabel('Frames')
    plt.ylabel('Winkel (in Grad)')
    plt.title(f'{axis}-Winkel der rechten Körperteile über Frames')

    # Füge eine Legende hinzu
    plt.legend()

    # Füge ein Gitter hinzu für bessere Lesbarkeit
    plt.grid(True, linestyle='--', alpha=0.7)

    # Zeige das Diagramm an
    plt.show()

# Beispielhafte Nutzung der Funktion
# Stelle sicher, dass dein 'angles'-Dictionary bereits definiert und gefüllt ist
# plot_right_angles(angles)
plot_right_angles(angles_z, "z")




In [ ]:
r_heel_num = 24
l_heel_num = 21

lower_leg_angles = {}


z_vector = np.array([0, 0, 1])
y_vector = np.array([0, 1, 0])
x_vector = np.array([1, 0, 0])

for i, frame in enumerate(list(result.keys())):
  lower_leg_angle = {}

  r_knee = result[frame]['3d_joints'][0][r_knee_num]
  l_knee = result[frame]['3d_joints'][0][l_knee_num]
  r_heel = result[frame]['3d_joints'][0][r_heel_num]
  l_heel = result[frame]['3d_joints'][0][l_heel_num]
  right_heel_knee_vector = r_knee - r_heel
  left_heel_knee_vector = l_knee - l_heel

  r_lower_leg_angle_x = np.degrees(np.arccos(np.dot(right_heel_knee_vector, x_vector) / (np.linalg.norm(right_heel_knee_vector) * np.linalg.norm(x_vector))))
  l_lower_leg_angle_x = np.degrees(np.arccos(np.dot(left_heel_knee_vector, x_vector) / (np.linalg.norm(left_heel_knee_vector) * np.linalg.norm(x_vector))))

  r_lower_leg_angle_y = np.degrees(np.arccos(np.dot(right_heel_knee_vector, y_vector) / (np.linalg.norm(right_heel_knee_vector) * np.linalg.norm(y_vector))))
  l_lower_leg_angle_y = np.degrees(np.arccos(np.dot(left_heel_knee_vector, y_vector) / (np.linalg.norm(left_heel_knee_vector) * np.linalg.norm(y_vector))))

  r_lower_leg_angle_z = np.degrees(np.arccos(np.dot(right_heel_knee_vector, z_vector) / (np.linalg.norm(right_heel_knee_vector) * np.linalg.norm(z_vector))))
  l_lower_leg_angle_z = np.degrees(np.arccos(np.dot(left_heel_knee_vector, z_vector) / (np.linalg.norm(left_heel_knee_vector) * np.linalg.norm(z_vector))))

  lower_leg_angle["r_lower_leg_angle_x"] = r_lower_leg_angle_x
  lower_leg_angle["l_lower_leg_angle_x"] = l_lower_leg_angle_x
  lower_leg_angle["r_lower_leg_angle_y"] = r_lower_leg_angle_y
  lower_leg_angle["l_lower_leg_angle_y"] = l_lower_leg_angle_y
  lower_leg_angle["r_lower_leg_angle_z"] = r_lower_leg_angle_z
  lower_leg_angle["l_lower_leg_angle_z"] = l_lower_leg_angle_z

  lower_leg_angles[i] = lower_leg_angle


print(lower_leg_angles[0])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_lower_leg_angles(lower_leg_angles):
    """
    Plottet die Winkel der Unterschenkel (rechts und links) zur x-, y- und z-Achse über alle Frames.

    Parameter:
    - lower_leg_angles (dict): Dictionary mit Frame-Nummern als Schlüsseln und
                               Dictionaries der Winkel als Werte.
    """
    frames = sorted(lower_leg_angles.keys())

    # Initialisiere Listen für jede Winkelart
    r_angle_x = []
    l_angle_x = []
    r_angle_y = []
    l_angle_y = []
    r_angle_z = []
    l_angle_z = []

    for frame in frames:
        angles = lower_leg_angles[frame]
        r_angle_x.append(angles.get("r_lower_leg_angle_x", np.nan))
        l_angle_x.append(angles.get("l_lower_leg_angle_x", np.nan))
        r_angle_y.append(angles.get("r_lower_leg_angle_y", np.nan))
        l_angle_y.append(angles.get("l_lower_leg_angle_y", np.nan))
        r_angle_z.append(angles.get("r_lower_leg_angle_z", np.nan))
        l_angle_z.append(angles.get("l_lower_leg_angle_z", np.nan))

    plt.figure(figsize=(14, 7))

    # Plot für rechte Unterschenkel
    plt.plot(frames, r_angle_x, label='Rechts - X-Achse', marker='o')
    plt.plot(frames, r_angle_y, label='Rechts - Y-Achse', marker='o')
    plt.plot(frames, r_angle_z, label='Rechts - Z-Achse', marker='o')

    # Plot für linke Unterschenkel
    plt.plot(frames, l_angle_x, label='Links - X-Achse', marker='s')
    plt.plot(frames, l_angle_y, label='Links - Y-Achse', marker='s')
    plt.plot(frames, l_angle_z, label='Links - Z-Achse', marker='s')

    plt.xlabel('Frames')
    plt.ylabel('Winkel (°)')
    plt.title('Unterschenkelwinkel zur x-, y- und z-Achse über Frames')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

# Beispielhafte Nutzung der Funktion
# Stelle sicher, dass dein 'lower_leg_angles'-Dictionary bereits definiert und gefüllt ist
# plot_lower_leg_angles(lower_leg_angles)

plot_lower_leg_angles(lower_leg_angles)


In [ ]:
COM_angles = {}

for i, frame in enumerate(list(result.keys())):
    COM_angle = {}

    # Berechne den Vektor vom Knöchel-Mittelpunkt zum COM
    ground_to_COM_vector = np.array(COM_xyz_values[i]) - np.array(center_ankle[i])

    # Berechne die Winkel zur x-, y- und z-Achse
    COM_x_rotation = np.degrees(np.arccos(np.dot(ground_to_COM_vector, x_vector) / (np.linalg.norm(ground_to_COM_vector) * np.linalg.norm(x_vector))))
    COM_y_rotation = np.degrees(np.arccos(np.dot(ground_to_COM_vector, y_vector) / (np.linalg.norm(ground_to_COM_vector) * np.linalg.norm(y_vector))))
    COM_z_rotation = np.degrees(np.arccos(np.dot(ground_to_COM_vector, z_vector) / (np.linalg.norm(ground_to_COM_vector) * np.linalg.norm(z_vector))))

    COM_angle["COM_x_rotation"] = COM_x_rotation
    COM_angle["COM_y_rotation"] = COM_y_rotation
    COM_angle["COM_z_rotation"] = COM_z_rotation

    COM_angles[i] = COM_angle

print(COM_angles[0])


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_COM_angles(COM_angles):
    """
    Plottet die COM-Winkel zur x-, y- und z-Achse über alle Frames.

    Parameter:
    - COM_angles (dict): Dictionary mit Frame-Nummern als Schlüsseln und
                         Dictionaries der Winkel als Werte.
                         Beispiel:
                         {
                             0: {"COM_x_rotation": 30.0, "COM_y_rotation": 45.0, "COM_z_rotation": 60.0},
                             1: {"COM_x_rotation": 32.0, "COM_y_rotation": 46.0, "COM_z_rotation": 61.0},
                             ...
                         }
    """
    frames = sorted(COM_angles.keys())

    # Listen der Winkelwerte
    COM_x = [COM_angles[frame].get("COM_x_rotation", np.nan) for frame in frames]
    COM_y = [COM_angles[frame].get("COM_y_rotation", np.nan) for frame in frames]
    COM_z = [COM_angles[frame].get("COM_z_rotation", np.nan) for frame in frames]

    plt.figure(figsize=(12, 6))

    # Plot für jede Achse
    plt.plot(frames, COM_x, label='COM X Rotation', marker='o')
    plt.plot(frames, COM_y, label='COM Y Rotation', marker='s')
    plt.plot(frames, COM_z, label='COM Z Rotation', marker='^')

    # Achsenbeschriftungen und Titel
    plt.xlabel('Frames')
    plt.ylabel('Winkel (°)')
    plt.title('COM Winkel zu den Achsen über Frames')

    # Legende und Gitter
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()

# Beispielhafte Nutzung der Funktion
# Stelle sicher, dass dein 'COM_angles'-Dictionary bereits definiert und gefüllt ist
plot_COM_angles(COM_angles)
